<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/for_prediction_use%20this_yeappppppy_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Load the data
df = pd.read_csv('stock_weekly_returns_test.csv')
df=df.dropna()
# Preprocess the data
df['date'] = pd.to_datetime(df['date'])
#df['outperform'] = (df['weekly_return'] > 0).astype(int)
# Encode ticker as numeric
'''
label_encoder = LabelEncoder()
df['ticker_encoded'] = label_encoder.fit_transform(df['ticker'])

# Select features for the model
features = ['ticker_encoded','dollar_volume', 'adj close', 'atr', 'bb_high', 'bb_low', 'bb_mid',
            'garman_klass_vol', 'macd', 'rsi', 'return_1m', 'return_2m',
            'return_3m', 'return_6m', 'return_9m', 'return_12m']

X = df[features]
y = df['outperform']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_scaled)

# Get probability estimates
y_prob = rf_classifier.predict_proba(X_test_scaled)

# Create a DataFrame with test data, predictions, and probabilities
results_df = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred,
    'prob_underperform': y_prob[:, 0],
    'prob_outperform': y_prob[:, 1],

})



# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)
# Add original features to the results DataFrame
results_df = pd.concat([results_df, X_test.reset_index(drop=True)], axis=1)

# Function to predict for new data
def predict_performance(new_data):
    new_data_scaled = scaler.transform(new_data)
    prediction = rf_classifier.predict_proba(new_data_scaled)
    return prediction
# Function to predict and rank stocks
'''

'\nlabel_encoder = LabelEncoder()\ndf[\'ticker_encoded\'] = label_encoder.fit_transform(df[\'ticker\'])\n\n# Select features for the model\nfeatures = [\'ticker_encoded\',\'dollar_volume\', \'adj close\', \'atr\', \'bb_high\', \'bb_low\', \'bb_mid\',\n            \'garman_klass_vol\', \'macd\', \'rsi\', \'return_1m\', \'return_2m\',\n            \'return_3m\', \'return_6m\', \'return_9m\', \'return_12m\']\n\nX = df[features]\ny = df[\'outperform\']\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Scale the features\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n\n# Create and train the Random Forest Classifier\nrf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)\nrf_classifier.fit(X_train_scaled, y_train)\n\n# Make predictions on the test set\ny_pred = rf_classifier.predict(X_test_scaled)\n\n# Get 

In [ ]:
import joblib


def save_model(model, scaler, label_encoder, filename='rf_model.joblib'):
    """
    Save the trained model, scaler, and label encoder to disk.

    Args:
    model (RandomForestClassifier): Trained Random Forest model
    scaler (StandardScaler): Fitted StandardScaler
    label_encoder (LabelEncoder): Fitted LabelEncoder
    filename (str): Name of the file to save the model (default: 'rf_model.joblib')
    """
    model_data = {
        'model': model,
        'scaler': scaler,
        'label_encoder': label_encoder
    }
    joblib.dump(model_data, filename)
    print(f"Model saved to {filename}")



In [ ]:
save_model(rf_classifier, scaler, label_encoder)

Model saved to rf_model.joblib


In [11]:
# needed in predictions file (yet to be created)
import joblib

def load_model_and_predict(filename='rf_model.joblib'):
    """
    Load the model from disk and use it to make predictions.

    Args:
    filename (str): Name of the file containing the saved model (default: 'rf_model.joblib')

    Returns:
    function: A function that takes new data and returns predictions
    """
    model_data = joblib.load(filename)
    model = model_data['model']
    scaler = model_data['scaler']
    label_encoder = model_data['label_encoder']

    def predict(new_data):
        """
        Make predictions using the loaded model.

        Args:
        new_data (pd.DataFrame): New data to make predictions on

        Returns:
        np.array: Probability estimates for each class
        """
        # Ensure the new data has the same features as the training data
        required_features = ['ticker', 'dollar_volume', 'adj close', 'atr', 'bb_high', 'bb_low', 'bb_mid',
                             'garman_klass_vol', 'macd', 'rsi', 'return_1m', 'return_2m',
                             'return_3m', 'return_6m', 'return_9m', 'return_12m']

        if not all(feature in new_data.columns for feature in required_features):
            raise ValueError("New data does not contain all required features")

        # Encode the ticker
        new_data['ticker_encoded'] = label_encoder.transform(new_data['ticker'])

        # Select and order the features
        X = new_data[['ticker_encoded', 'dollar_volume', 'adj close', 'atr', 'bb_high', 'bb_low', 'bb_mid',
                      'garman_klass_vol', 'macd', 'rsi', 'return_1m', 'return_2m',
                      'return_3m', 'return_6m', 'return_9m', 'return_12m']]

        # Scale the features
        X_scaled = scaler.transform(X)

        # Make predictions
        predictions = model.predict_proba(X_scaled)
        results_df = pd.DataFrame({
            'Date':new_data['date'],
            'prob_underperform': predictions[:, 0],
            'prob_outperform': predictions[:, 1],
            })
        results_df['ticker_name']=label_encoder.inverse_transform(new_data['ticker_encoded'].astype(int))
        return results_df

    print(f"Model loaded from {filename}")
    return predict

# Example usage:

predict_fn = load_model_and_predict()
#new_data = pd.DataFrame(...)  # Your new data here
predictions = predict_fn(df)


Model loaded from rf_model.joblib


In [12]:
predictions

,Date,prob_underperform,prob_outperform,ticker_name
0,2023-12-31,0.49,0.51,ADANIENT.NS
1,2024-01-07,0.39,0.61,ADANIENT.NS
2,2024-01-14,0.35,0.65,ADANIENT.NS
3,2024-01-21,0.38,0.62,ADANIENT.NS
4,2024-01-28,0.46,0.54,ADANIENT.NS
...,...,...,...,...
1695,2024-07-21,0.50,0.50,WIPRO.NS
1696,2024-07-28,0.38,0.62,WIPRO.NS
1697,2024-08-04,0.38,0.62,WIPRO.NS
1698,2024-08-11,0.47,0.53,WIPRO.NS


In [13]:
predictions.to_csv('results.csv', index=False)